In [1]:
import CharacterizingAffineCSemigroup
from CharacterizingAffineCSemigroup import *

In [2]:
gen0 = [[4,0],[3,0],[5,0],[5,5],[2,2],[3,3],[4,3],[5,3],[5,1],[6,1],[7,1],[5,4]]

In [3]:
IsCsemigroup(gen0)

True

In [4]:
C0 = Cone(cone=gen0)
rayos0 = C0.ExtremeRays()
rayos0

[[1, 0], [1, 1]]

In [5]:
# Calculamos la multiplicidad en cada rayo.
# INPUT:
#   - ray: rayo del cono.
#   - smg: sistema minimal de generadores.
# OUTPUT:
#   - Elemento minimal en el rayo.
def MultiplicityAxis(ray,smg):
    multiplicity = []
    for x in smg:
        aux = BelongAxis(x,ray)
        if aux != 0:
            multiplicity.append([aux,x])
    return sorted(multiplicity)[0][1]

In [6]:
MultiplicityAxis([1,0],gen0)

[3, 0]

In [7]:
# Calculamos el diamante entero de las multiplicidades.
# INPUT:
#   - cone: rayos del cono.
#   - smg: sistema minimal de generadores.
# OUTPUT:
#   - diamante entero de las multiplicidades
def DiamondMultiplicity(cone,smg):
    extremos = []
    for ray in cone:
        extremos.append(MultiplicityAxis(ray,smg))
    diamante = Diamond(extremos)
    hull = ConvexHull(diamante)
    eq = [list(x) for x in hull.equations]
    cotas = Cube(diamante)
    return IntegerDiamond(eq,cotas)

In [8]:
d=DiamondMultiplicity([[1,0],[1,1]],gen0)

In [9]:
# Ecuaciones de un rayo.
eq = EqRay([1,0],[2,1])

In [10]:
# Borramos elementos repetidos.
# INPUT:
#   - v: vector de entrada.
# OUTPUT:
#   - vector v sin elementos de entrada.
def DeleteDuplicates(v):
    w = []
    for i in range(len(v)):
        if v[i] not in w:
            w.append(v[i])
    return w

In [11]:
DeleteDuplicates(AffineTerm(eq,d))

[[1], [2]]

In [12]:
# Creamos una función que calcula la uno norma.
# INPUT:
#   - v: un vector.
# OUTPUT:
#   - Norma-1 del vector v.
def NormOne(v):
    suma = 0
    for x in v:
        suma = suma + x
    return suma

In [13]:
NormOne([3,4,6,8,0])

21

In [14]:
# Calculamos el punto de menor norma del diamante para una recta afín.
# INPUT:
#   - diamond: diamante entero.
#   - afin: término afin de la recta.
#   - eqray: ecuaciones de la recta.
# OUTPUT:
#   - Punto de menor norma del diamante que pasa por la recta afín.
def MinimumPointAffineDiamond(diamond,afin,eqray):
    numeq = len(eqray)
    aux2 = []
    for d in diamond:
        aux = MultiplyMatrix(eqray,[[x] for x in d])
        if aux == [afin]:
            aux2.append([NormOne(d),d])
    return(sorted(aux2)[0][1])

In [15]:
MinimumPointAffineDiamond(d,[1],eq)

[1, 1]

In [16]:
# Calculamos los elementos de norma mínima por cada recta afín.
# INPUT:
#   - vdir: vector director de la recta.
#   - seed: punto para empezar a mirar la recta.
#   - smgen: sistema minimal de generadores.
# OUTPUT:
#   - Norma mínima de los elementos del semigrupo de la recta. 
def ComputeMinimumNormInSemigroupLine(vdir,seed,smgen):
    v = seed
    afseg = AffineSemigroup(smgen, "generators")
    while True:
        if afseg.belongs(v):
            return NormOne(v)
        v = [v[i] + seed[i] for i in range(len(vdir))]

In [17]:
ComputeMinimumNormInSemigroupLine([1,0],[1,1],gen0)

4

In [26]:
# Calcula un valor en el rayo y en el semigrupo con norma mayor que una dada.
# INPUT:
#   - n: cota de la norma.
#   - ray: rayo del cono.
#   - gen: generadores del semigrupo.
# OUTPUT:
#   - Valor en el rayo que está en el semigrupo y tiene norma mayor que n.
def ComputeBoundRay(n,ray,gen):
    afseg = AffineSemigroup(gen, "generators")
    v = ray
    i = 1
    while True:
        v = [i*x for x in ray]
        if NormOne(v) > n and afseg.belongs(v):
            return v
        i = i+1

In [27]:
sgaf = AffineSemigroup(gen0, "generators")

In [28]:
AffineSemigroup(gen0, "generators").getMSG()

[[5, 1], [5, 4], [3, 3], [7, 1], [3, 0], [6, 1], [5, 0], [2, 2], [4, 3], [5, 3
], [4, 0]]

In [29]:
ComputeBoundRay(7,[1,0],gen0)

[8, 0]

In [41]:
def ComputeGaps(gen):
    # Calculamos el cono.
    C = Cone(cone=gen)
    # Calculamos los rayos del cono.
    rayos = C.ExtremeRays()
    # Calculamos del diamante de multiplicidades.
    diamanteM = DiamondMultiplicity(rayos,gen)
    # Elegimos un punto en el interior del diamante.
    dimension = len(rayos[0])
    calibre  = [0 for x in range(dimension)]
    for ray in rayos:
        calibre = [calibre[i]+ray[i] for i in range(dimension)]
    x = []
    # Para cada rayo del cono.
    for ray in rayos:
        # Calculamos el conductor.
        conductor = ConductorAxis(gen,ray)
        # Calculamos las ecuaciones del rayo.
        eq = EqRay(ray,calibre)
        # Calculamos los términos afines.
        afinesDiamante = DeleteDuplicates(AffineTerm(eq,diamanteM))
        # Calculamos la máxima norma mínima en cada recta afin.
        maxMinNorm = 0
        for afin in afinesDiamante:
            # En primer lugar miramos la norma minima en el diamante.
            minimumNormDiamond = MinimumPointAffineDiamond(diamanteM,afin,eq)
            # Ahora buscamos el elemento mínimo en el diamante en dicha recta.
            minimumNormSG = ComputeMinimumNormInSemigroupLine(ray,minimumNormDiamond,gen)
            if maxMinNorm < minimumNormSG:
                maxMinNorm = minimumNormSG
        n = NormOne(conductor) + maxMinNorm
        x.append(ComputeBoundRay(n,ray,gen0))
    # Calculamos el diamante de esos valores X
    diamanteX = Diamond(x)
    # Creamos un cubo que contenga al diamante.
    cotaX = Cube(diamanteX)
    # Calculamos las ecuaciones del diamante.
    hullX = ConvexHull(diamanteX)
    eqDiamanteX = [list(x) for x in hullX.equations]
    # Calculamos el diamante entero X.
    diamanteEnteroX = IntegerDiamond(eqDiamanteX,cotaX)
    # Inicialmente el conjunto de huecos es vacío.
    gapset = []
    # Definimos el semigrupo afín.
    afseg = AffineSemigroup(gen, "generators")
    for x in diamanteEnteroX:
        if not afseg.belongs(x):
            gapset.append(x)
    return gapset

In [42]:
ComputeGaps(gen0)

[[1, 0], [1, 1], [2, 0], [2, 1], [3, 1], [3, 2], [4, 1], [4, 2], [6, 4]]